> ### **Async Disposable**:
>    Version *C# 8.0*

> The **DisposeAsync()** method of the **IAsyncDisposable** Interface in C#  
>
> The public parameterless DisposeAsync() method is called automatically in an await using statement, and the purpose of this DisposeAsync() method is to free the unmanaged resources.   
>
>
> Freeing the memory associated with a managed object is always the responsibility of the garbage collector. Because of this, it has a standard implementation as follows:  

```csharp
    public async ValueTask DisposeAsync(){
        // Perform async clean-up.
        await DisposeAsyncCore();

        // Dispose of unmanaged resources.
        Dispose(false);

        // Dispose methods should call SuppressFinalize
        // Suppress finalization.
        GC.SuppressFinalize(this);
    }
```

> Implement the **DisposeAsync** Method of the **IAsyncDisposable** Interface in C#
>
>  - The *IAsyncDisposable* interface was introduced as part of C# 8.0.  
>
>  - We need to implement the *DisposeAsync()* method when we want to perform resource clean-up in the same way as we used to by implementing the *Dispose()* method of *IDisposable* Interface.     
>
>  - The difference between these two interfaces is that the *DisposeAsync()* method allows for asynchronous clean-up operations whereas the *Dispose()* method performs synchronous clean-up operations.   
>
>  - The *DisposeAsync()* method of the *IAsyncDisposable* interface returns a ValueTask that represents the asynchronous dispose of operation. 
>
>
```csharp
    public async ValueTask DisposeAsync()
    {
        // Perform async clean-up.
        await DisposeAsyncCore();
        // Dispose of unmanaged resources.
        Dispose(false);
        // Dispose methods should call SuppressFinalize
        // Suppress finalization.
        GC.SuppressFinalize(this);
    }
```

> **The DisposeAsyncCore() Method**
> 
>
>  - The *DisposeAsyncCore()* method is intended to perform the asynchronous clean-up of managed resources.
>
>  - It encapsulates the common asynchronous clean-up operations when a subclass inherits a base class that is an implementation of *IAsyncDisposable*.
>
>  - The *DisposeAsyncCore()* method is virtual so that derived classes can define additional clean-up by overriding this method.
>
>  - If an implementation of IAsyncDisposable is sealed, the *DisposeAsyncCore()* method is not needed, and the asynchronous clean-up can be performed directly in the *DisposeAsync()* method.

> **Simple example**:
>
>  This example shows how to create a class called Sample that implements the IAsyncDisposable interface and provides the implementation for the DisposeAsync method.
>
>  This method performs some asynchronous clean-up operations and disposes of an unmanaged resource (a TextWriter object).

In [2]:
using System;
using System.IO;
using System.Threading.Tasks;

public class SampleTextWriter : IAsyncDisposable
{
    private TextWriter _writer;
    public SampleTextWriter(string filePath)
    {
        _writer = File.CreateText(filePath);
    }

    public async ValueTask DisposeAsync()
    {
        // Perform async clean-up.
        await _writer.WriteLineAsync("Async disposal!");
        await Task.Delay(1000);

        // Dispose of unmanaged resources.
        _writer.Dispose();

        // Suppress finalization.
        GC.SuppressFinalize(this);
    }
}
        await using (var sample = new SampleTextWriter("test.txt"))  // DisposeAsync is called implicitly at the end of the await using block.
        {
            // Do some work with sample.
        }

> **Inheritance example**:
>
> - This example shows how to create a base class called Sample that implements the IAsyncDisposable interface and provides a virtual method called DisposeAsyncCore for subclasses to override.   
>
>  - The base class also implements the IDisposable interface for synchronous disposal.
>
>  - The subclass called SampleInherited overrides the DisposeAsyncCore method and performs some additional asynchronous clean-up operations.

In [ ]:
using System;
using System.IO;
using System.Threading.Tasks;

public class SampleTextWriter : IAsyncDisposable, IDisposable
{
    private TextWriter _writer;

    public SampleTextWriter(string filePath)
    {
        _writer = File.CreateText(filePath);
    }

    public async ValueTask DisposeAsync()
    {
        // Perform async clean-up.
        await DisposeAsyncCore();

        // Dispose of unmanaged resources.
        Dispose(false);

        // Suppress finalization.
        GC.SuppressFinalize(this);
    }

    protected virtual async ValueTask DisposeAsyncCore()
    {
        // Perform async clean-up of managed resources.
        await _writer.WriteLineAsync("Async disposal from base class!");
        await Task.Delay(1000);
    }

    public void Dispose()
    {
        // Dispose of unmanaged resources.
        Dispose(true);

        // Suppress finalization.
        GC.SuppressFinalize(this);
    }

    protected virtual void Dispose(bool disposing)
    {
        if (disposing)
        {
            // Dispose of managed resources.
            _writer.Dispose();
        }
    }
}

public class ExtendedTextWriter : SampleTextWriter
{
    public ExtendedTextWriter(string filePath) : base(filePath)
    {
    }

    protected override async ValueTask DisposeAsyncCore()
    {
        // Perform async clean-up of subclass resources.
        await base.DisposeAsyncCore();
        await Console.Out.WriteLineAsync("Async disposal from subclass!");
        await Task.Delay(1000);
    }
}


        await using (var sample = new ExtendedTextWriter("test.txt"))
        {
            // Do some work with sample.
        }
        // DisposeAsync is called implicitly at the end of the await using block.

> **Example:1**
> 
>   Using a database connection to execute a query asynchronously: 
>    - A database connection is another class that implements both IDisposable and IAsyncDisposable interfaces, so it can be used with either using or await using statements.
>    - The following code snippet shows how to use a database connection to execute a query asynchronously and display the result on the console.

In [ ]:
using System;
using System.IO;
using System.Threading.Tasks;


// Use await using to dispose the stream reader asynchronously
await using (var reader = new StreamReader("sample.txt"))
{
    string line;
    // Read the file line by line asynchronously
    while ((line = await reader.ReadLineAsync()) != null)
    {
        // Display the line on the console
        Console.WriteLine(line);
    }
}

> **Example:2**
> 
>Using a database connection to execute a query asynchronously: 
> - A database connection is another class that implements both IDisposable and IAsyncDisposable interfaces, so it can be used with either using or await using statements.
> - The following code snippet shows how to use a database connection to execute a query asynchronously and display the result on the console.

In [ ]:
using System;
using System.Data.SqlClient;
using System.Threading.Tasks;


// Use await using to dispose the database connection asynchronously
await using (var connection = new SqlConnection("connection string"))
{
    // Open the connection asynchronously
    await connection.OpenAsync();
    // Create a command to execute a query asynchronously
    using var command = new SqlCommand("select * from table", connection);
    // Execute the query and get a data reader asynchronously
    using var reader = await command.ExecuteReaderAsync();
    // Read the data reader and display the result on the console
    while (await reader.ReadAsync())
    {
        Console.WriteLine(reader[0]);
    }
}

# Continue learning

There are plenty more resources out there to learn!

> [⏩ Next Module - Indices and Ranges](55.IndicesAndRanges.ipynb)
>
> [⏪ Last Module - Async Streams](54.0.AsyncStreams.Ipynb)
>
> [Reference - asynchronous disposable in csharp-8](https://dotnettutorials.net/lesson/asynchronous-disposable-in-csharp-8/)